In [36]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import tensorflow as tf
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

In [37]:
from tensorflow.examples.tutorials.mnist import input_data
#get mnist data, with one_hot encoding
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
#suppress warnings
tf.logging.set_verbosity(old_v)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


<h2>Define hyperparameters</h2>

In [38]:
#you can print out the label for 1st figure
print(mnist.train.labels[0])

num_train = mnist.train.num_examples #55,000
num_validation = mnist.validation.num_examples #5000
num_test = mnist.test.num_examples #10,000

#learning rate
lr = 0.005
#number of traning steps
num_steps =5000
#number of batch_size
batch_size = 128

#network parameters
n_hidden_1 = 300
n_hidden_2 = 100
# n_hidden_3 = 100
# n_hidden_4 = 50
num_input = 784
num_classes = 10

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


<h2>Define placeholder and Variables</h2>

In [39]:
tf.reset_default_graph()

#tf graph input
keep_prob = tf.placeholder(tf.float32)
X = tf.placeholder(tf.float32,[None,num_input],name='X')
Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')

#Layers weight & bias
weights = {
    'W1': tf.Variable(tf.random_normal([num_input, n_hidden_1]),name='W1'),
    'W2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]),name='W2'),
#     'W3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3]),name='W3'),
#     'W4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4]),name='W4'),
    'Wout': tf.Variable(tf.random_normal([n_hidden_2, num_classes]),name='Wout')
}

biases = {
    'b1': tf.Variable(tf.zeros(shape=[n_hidden_1]),name='b1'),
    'b2': tf.Variable(tf.zeros(shape=[n_hidden_2]),name='b2'),
#     'b3': tf.Variable(tf.zeros(shape=[n_hidden_3]),name='b3'),
#     'b4': tf.Variable(tf.zeros(shape=[n_hidden_4]),name='b4'),
    'bout': tf.Variable(tf.zeros(shape=[num_classes]),name='bout')
}

<h2>Define neural network</h2>

In [40]:
#define a neural net model
def neural_net(x, activation_function):
    if activation_function is None:
        layer_1_out = tf.add(tf.matmul(x,weights['W1']),biases['b1'])
        layer_2_out = tf.add(tf.matmul(layer_1_out,weights['W2']),biases['b2'])
#         layer_3_out = tf.add(tf.matmul(layer_2_out,weights['W3']),biases['b3'])
#     layer_4_out = tf.add(tf.matmul(layer_3_out,weights['W4']),biases['b4'])
        out = tf.add(tf.matmul(layer_2_out,weights['Wout']),biases['bout'])  #output layer no need activation
        out = tf.nn.dropout(out, keep_prob)
    else:
        layer_1_out = activation_function(tf.add(tf.matmul(x,weights['W1']),biases['b1']))
        layer_2_out = activation_function(tf.add(tf.matmul(layer_1_out,weights['W2']),biases['b2']))
        out = tf.add(tf.matmul(layer_2_out,weights['Wout']),biases['bout'])
        out = tf.nn.dropout(out, keep_prob)
    return out

<h2>Define cost function and accuracy</h2>

In [41]:
#predicted labels
logits = neural_net(X,activation_function=tf.nn.softmax)

#define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='loss')
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='loss')
#define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

#compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))

#compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

#Initialize the variables
init = tf.global_variables_initializer()

<h2>Execute training</h2>

In [42]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)  
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))  
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(6):
        for i in range(num_steps):
            #fetch batch
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            #run optimization
            sess.run(train_op, feed_dict={X:batch_x, Y:batch_y, keep_prob: 0.9})
            if i % 100 ==0:
                acc = sess.run(accuracy,feed_dict={X:batch_x, Y:batch_y, keep_prob: 1})
                print("step "+str(i)+", Accuracy= {:.3f}".format(acc))
    
    print("Training finished!")
    
    print("Testing ACcuracy:", sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels, keep_prob: 1}))

step 0, Accuracy= 0.062
step 100, Accuracy= 0.547
step 200, Accuracy= 0.719
step 300, Accuracy= 0.852
step 400, Accuracy= 0.859
step 500, Accuracy= 0.836
step 600, Accuracy= 0.875
step 700, Accuracy= 0.898
step 800, Accuracy= 0.891
step 900, Accuracy= 0.844
step 1000, Accuracy= 0.867
step 1100, Accuracy= 0.883
step 1200, Accuracy= 0.914
step 1300, Accuracy= 0.930
step 1400, Accuracy= 0.930
step 1500, Accuracy= 0.891
step 1600, Accuracy= 0.961
step 1700, Accuracy= 0.945
step 1800, Accuracy= 0.953
step 1900, Accuracy= 0.945
step 2000, Accuracy= 0.977
step 2100, Accuracy= 0.930
step 2200, Accuracy= 0.945
step 2300, Accuracy= 0.977
step 2400, Accuracy= 0.930
step 2500, Accuracy= 0.938
step 2600, Accuracy= 0.914
step 2700, Accuracy= 0.961
step 2800, Accuracy= 0.953
step 2900, Accuracy= 0.930
step 3000, Accuracy= 0.969
step 3100, Accuracy= 0.961
step 3200, Accuracy= 0.945
step 3300, Accuracy= 0.953
step 3400, Accuracy= 0.953
step 3500, Accuracy= 0.945
step 3600, Accuracy= 0.945
step 3700, Ac

<h2>Your results</h2>

In [ ]:
Learning rate and number of epochs contribute the most to improvement in performance.
Number of neurous and layers do change it. Since each problem has a corresponding complexity, like a bootleneck, if these numbers exceed a certain threshold, the performance will not change much.
Some tricks can improve a little bit, like optimization.